# General material models

In [1]:
import numpy as np

from sigmaepsilon.solid.material import ElasticityTensor
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
from sigmaepsilon.math.logical import isposdef, issymmetric
from sigmaepsilon.solid.material import (
    ElasticityTensor,
    LinearElasticMaterial,
    HuberMisesHenckyFailureCriterion,
)

hooke = elastic_stiffness_matrix(E=210000, NU=0.3)
assert isposdef(hooke)
assert issymmetric(hooke, 1e-8)

yield_strength=355.0

frame = ReferenceFrame(dim=3)
stiffness = ElasticityTensor(hooke, frame=frame, tensorial=False)
failure_model = HuberMisesHenckyFailureCriterion(yield_strength=yield_strength)
material = LinearElasticMaterial(stiffness=stiffness, failure_model=failure_model)

In [2]:
material.calculate_strains(np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0]))

array([ 0.00169048, -0.00050714, -0.00050714,  0.        ,  0.        ,
        0.        ])

In [3]:
material.utilization(stresses=np.random.rand(10, 6))

array([0.00284265, 0.00374942, 0.0057942 , 0.00408893, 0.0063019 ,
       0.00325941, 0.00287428, 0.00484382, 0.00455315, 0.00415884])

In [4]:
material.calculate_stresses(strains=np.random.rand(6)).shape

(6,)

In [5]:
material.calculate_stresses(strains=np.random.rand(10, 6)).shape

(10, 6)

In [6]:
stresses=np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0])
strains = material.calculate_strains(stresses=stresses)
material.utilization(stresses=stresses) * 100

100.0

In [7]:
strains = material.calculate_strains(stresses=np.eye(6) * yield_strength)
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

array([1.        , 1.        , 1.        , 1.73205081, 1.73205081,
       1.73205081])

In [8]:
strains = material.calculate_strains(stresses=np.array([10*yield_strength, 10*yield_strength, 10*yield_strength, 0.0, 0.0, 0.0]))
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

array(2.56195691e-15)

Utilization for a single state of strain:

In [9]:
strains = 2*np.random.rand(6)/1000
strains

array([0.00163524, 0.00134537, 0.00118588, 0.00032621, 0.00088649,
       0.00123073])

In [10]:
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses) * 100

63.720156267084384

Utilizations for several states with bulk evaluation:

In [11]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000) * 100

array([0.00066279, 0.00084289, 0.00092441, 0.00109238, 0.00087304,
       0.00063178, 0.00092051, 0.00101957, 0.00053558, 0.00065827])

In [12]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000).shape

(10,)

In [13]:
material.utilization(stresses=tuple(2*np.random.rand(10)/1000 for _ in range(6))) * 100

array([0.00116298, 0.00054026, 0.00110628, 0.00089284, 0.0013587 ,
       0.00043765, 0.000808  , 0.00100828, 0.00075597, 0.00100939])